In [1]:
import os

In [2]:
%pwd

'/Users/shrikanthnallapati/Projects/mlproject2/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/shrikanthnallapati/Projects/mlproject2'

In [5]:
# Update Entity
from dataclasses import dataclass
from pathlib import Path


# Entity _ which return type should the function return.
# custom return type other than configbox

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlProject2.constants import *
from mlProject2.utils.common import read_yaml, create_directories
 


In [11]:
# Update configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion # storing the data_ingestion config into into variable config

        create_directories([config.root_dir]) # create root directory for the variable config 

         # Returning the data type as defined in the DataIngestionCofig in the entity
#        # reading one by one from the config variable and storing them to assigned variables like root_dir, source_URL etc. 


        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
# update components
import os
import urllib.request as request
import zipfile
from src.mlProject2 import logger
from src.mlProject2.utils.common import get_size
# from src.mlProject2.entity.config_entity import DataIngestionConfig
from pathlib import Path

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [12]:
#update pipeline
try:
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()
        
except Exception as e:
    logger.exception(e)
    raise e



[2023-09-18 13:54:23,792: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-18 13:54:23,793: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-18 13:54:23,795: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-18 13:54:23,795: INFO: common: created directory at: artifacts]
[2023-09-18 13:54:23,796: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-18 13:54:24,296: INFO: 798708902: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26129
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e57c8b754d15353f0defdf9c702e8c0927ae4c0f592773ea7d498a45e6b95bc9"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B696:033B:C6FD97:F24C44:65088ECF
Accept-Ranges: bytes
Date: Mon, 18 Se